### 📚 __Introduction__

You are an analyst at a large online store. Together with the marketing department, you have compiled a list of hypotheses that could help increase revenue. You need to prioritize these hypotheses, run an A/B test, and analyze the results.

__Data Description__   

/datasets/hypotheses_us.csv

- Hypotheses: Brief descriptions of the hypotheses.
- Reach: User reach, on a scale of one to ten.
- Impact: Impact on users, on a scale of one to ten.
- Confidence: Confidence in the hypothesis, on a scale of one to ten.
- Effort: The resources required to test a hypothesis, on a scale of one to ten. The higher the Effort value, the more resources the test requires.

/datasets/orders_us.csv

- transactionId: Order ID.
- visitorId: ID of the user who placed the order.
- date: order date.
- revenue: order revenue.
- group: the A/B test group to which the user belongs.

/datasets/visits_us.csv

- date: the date.
- group: A/B test group.
- visits: the number of visits on the specified date in the specified A/B test group.

Make sure to preprocess your data. There may be errors in the original datasets; for example, some visitors may have been in both group A and group B.

__Part 1__. _Prioritize Hypotheses_
The hypotheses_us.csv file contains nine hypotheses about how to increase revenue for an online store, with _Reach, Impact, Confidence, and Effort_ specified for each.

The exercise consists:

- Apply the ICE framework to prioritize hypotheses. Order them in descending order of priority.
- Apply the RICE framework to prioritize hypotheses. Order them in descending order of priority.
- Show how hypothesis prioritization changes when you use RICE instead of ICE. Provide an explanation of the changes.

__Part 2__. _A/B Test Analysis_
You ran an A/B test and obtained the results described in the orders_us.csv and visits_us.csv files.

### 💻 __1. Notebook Libraries and Customization__

In [82]:
from IPython.display import display, HTML
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import re
from scipy import stats as st
from scipy.stats import gaussian_kde
from statsmodels.stats.proportion import proportions_ztest
import seaborn as sns
from tqdm import tqdm

pd.set_option('display.max_rows', 25)  # display max 25 rows
pd.set_option('display.max_columns', 25)  # display max 10 columns
# show max 15 characters in each column
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.width', 150)  # Display max 150 characters in cell

### 💻 __2. Functions__

In [2]:
# Function used to detect missing values ​​not treated as NAN due to the use of keep_defaukt_na=False
def missing_values_check(df):

    missing_values = ['', ' ', 'N/A', 'none', 'None',
                      'null', 'NULL', 'NaN', 'nan', 'NAN', 'nat', 'NaT']

    for column in df.columns:

        if df[column].isin(missing_values).any():

            display(HTML(
                f"> Null values ​​in column <i>'{column}'</i>: <b>{df[column].isin(missing_values).sum()}</b>"))

        else:

            display(
                HTML(f"> No null values ​​exist in column <i>'{column}'</i>"))
            
# Function used to assign pd.NA to missing values
def missing_values_replace(df):

    missing_values = ['', ' ', 'N/A', 'none', 'None',
                      'null', 'NULL', 'NaN', 'nan', 'NAN', 'nat', 'NaT']

    for column in df.columns:

        try:

            # (None, np.nan, pd.NA, pd.NaT)
            df[column] = df[column].apply(lambda x: pd.NA if pd.isna(x) else x)

            # (missing_values))
            df[column] = df[column].replace(missing_values, pd.NA)

        except Exception as e:

            display(
                HTML(f"> Column <i>'{column}'</i> could not be processed: {e}"))

    return df

def ice_prioritization(df, method="ease"):

    # Input validation
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame.")

    # Define columns according to method chosen
    if method == "effort":
        required_cols = {"impact", "confidence", "effort"}
        score_col = "ice_score"
    elif method == "ease":
        required_cols = {"impact", "confidence", "ease"}
        score_col = "ice_score"
    elif method == "rice":
        required_cols = {"reach", "impact", "confidence", "effort"}
        score_col = "rice_score"
    else:
        raise ValueError("Invalid method. Use 'effort', 'ease', or 'rice'.")

    # Required columns validation
    missing_cols = required_cols - set(df.columns)
    if missing_cols:
        raise ValueError(f"Missing required columns for {method} method: {missing_cols}")

    # Null values validation
    if df[list(required_cols)].isnull().any().any():
        raise ValueError(f"Columns {required_cols} must not contain null values.")

    # Guard rails for division
    if method in {"effort", "rice"} and (df["effort"] == 0).any():
        raise ZeroDivisionError("Column 'effort' contains zero(s); cannot divide by zero.")
    
    # Optional: ensure numeric types (will raise if non-numeric present)
    numeric_check = df[list(required_cols)].apply(pd.to_numeric, errors="raise")

    # Copy DataFrame
    df = df.copy()

    # Calcular ICE/RICE según método
    if method == "effort":
        df[score_col] = (df["impact"] * df["confidence"]) / df["effort"]
    elif method == "ease":
        df[score_col] = df["impact"] * df["confidence"] * df["ease"]
    else:  # "rice"
        df[score_col] = (df["reach"] * df["impact"] * df["confidence"]) / df["effort"]

    return df

### 🔁 __3. Data Loading__

In [3]:
df_hypotheses = pd.read_csv('../data/raw/hypotheses_us.csv', sep=';', header='infer', keep_default_na=False)
df_orders = pd.read_csv('../data/raw/orders_us.csv', sep=',', header='infer', keep_default_na=False)
df_visits = pd.read_csv('../data/raw/visits_us.csv', sep=',', header='infer', keep_default_na=False)

### 🧹 __4. Data Cleanup__

##### **4.1** Data Overview

In [4]:
df_hypotheses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Hypothesis  9 non-null      object
 1   Reach       9 non-null      int64 
 2   Impact      9 non-null      int64 
 3   Confidence  9 non-null      int64 
 4   Effort      9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 492.0+ bytes


In [5]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   transactionId  1197 non-null   int64  
 1   visitorId      1197 non-null   int64  
 2   date           1197 non-null   object 
 3   revenue        1197 non-null   float64
 4   group          1197 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 46.9+ KB


In [6]:
df_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    62 non-null     object
 1   group   62 non-null     object
 2   visits  62 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


##### **4.2** Standardization of column heading formats (Lower case, snake case)

In [7]:
df_hypotheses.columns = df_hypotheses.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
df_hypotheses.columns

Index(['hypothesis', 'reach', 'impact', 'confidence', 'effort'], dtype='object')

In [8]:
df_orders.columns = df_orders.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
df_orders.columns

Index(['transactionid', 'visitorid', 'date', 'revenue', 'group'], dtype='object')

In [9]:
df_visits.columns = df_visits.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
df_visits.columns

Index(['date', 'group', 'visits'], dtype='object')

##### **4.2** Explicit Duplicate Removal

In [10]:
display(HTML(f"> <b>df_hypotheses</b> explicit duplicates: {df_hypotheses.duplicated().sum()}"))

In [11]:
display(HTML(f"> <b>df_orders</b> explicit duplicates: {df_orders.duplicated().sum()}"))

In [12]:
display(HTML(f"> <b>df_visits</b> explicit duplicates: {df_visits.duplicated().sum()}"))

##### **4.3** Missing Value Analysis

In [13]:
missing_values_check(df_hypotheses)

In [14]:
missing_values_check(df_orders)

In [15]:
missing_values_check(df_visits)

##### **4.4** Casting Datatypes

In [16]:
df_hypotheses['hypothesis'] = df_hypotheses['hypothesis'].astype('string')
display(HTML(f"> df_hypotheses column: <b>'hypothesis'</b> data type: <i>{df_hypotheses['hypothesis'].dtypes}</i>"))
df_hypotheses

,hypothesis,reach,impact,confidence,effort
0,Add two new channels for attracting traffic. T...,3,10,8,6
1,Launch your own delivery service. This will sh...,2,5,4,10
2,Add product recommendation blocks to the store...,8,3,7,3
3,Change the category structure. This will incre...,8,3,3,8
4,Change the background color on the main page. ...,3,1,1,1
5,Add a customer review page. This will increase...,3,2,2,3
6,Show banners with current offers and sales on ...,5,3,8,3
7,Add a subscription form to all the main pages....,10,7,8,5
8,Launch a promotion that gives users discounts ...,1,9,9,5


In [17]:
df_orders['date'] = pd.to_datetime(df_orders['date']).dt.date
display(HTML(f"> df_orders column: <b>'date'</b> data type: <i>{df_orders['date'].dtypes}</i>"))
df_orders

,transactionid,visitorid,date,revenue,group
0,3667963787,3312258926,2019-08-15,30.4,B
1,2804400009,3642806036,2019-08-15,15.2,B
2,2961555356,4069496402,2019-08-15,10.2,A
3,3797467345,1196621759,2019-08-15,155.1,B
4,2282983706,2322279887,2019-08-15,40.5,B
...,...,...,...,...,...
1192,2662137336,3733762160,2019-08-14,100.8,B
1193,2203539145,370388673,2019-08-14,50.1,A
1194,1807773912,573423106,2019-08-14,165.3,A
1195,1947021204,1614305549,2019-08-14,5.5,A


In [18]:
df_orders['group'] = df_orders['group'].astype('category')
display(HTML(f"> df_orders column: <b>'group'</b> data type: <i>{df_orders['group'].dtypes}</i>"))
df_orders

,transactionid,visitorid,date,revenue,group
0,3667963787,3312258926,2019-08-15,30.4,B
1,2804400009,3642806036,2019-08-15,15.2,B
2,2961555356,4069496402,2019-08-15,10.2,A
3,3797467345,1196621759,2019-08-15,155.1,B
4,2282983706,2322279887,2019-08-15,40.5,B
...,...,...,...,...,...
1192,2662137336,3733762160,2019-08-14,100.8,B
1193,2203539145,370388673,2019-08-14,50.1,A
1194,1807773912,573423106,2019-08-14,165.3,A
1195,1947021204,1614305549,2019-08-14,5.5,A


In [19]:
df_visits['date'] = pd.to_datetime(df_visits['date']).dt.date
display(HTML(f"> df_visits column: <b>'date'</b> data type: <i>{df_visits['date'].dtypes}</i>"))
df_visits

,date,group,visits
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756
...,...,...,...
57,2019-08-27,B,720
58,2019-08-28,B,654
59,2019-08-29,B,531
60,2019-08-30,B,490


In [20]:
df_visits['group'] = df_visits['group'].astype('category')
display(HTML(f"> df_visits column: <b>'group'</b> data type: <i>{df_visits['group'].dtypes}</i>"))
df_visits

,date,group,visits
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756
...,...,...,...
57,2019-08-27,B,720
58,2019-08-28,B,654
59,2019-08-29,B,531
60,2019-08-30,B,490


##### **4.4** Handling Errors in Datasets (Same visitorid appears in both groups(A and B))

`LSPL` - __Strict rule:__ (most commonly used in industry):

The user appears in both groups → they are completely eliminated from A and B.   
Reason: You can't be 100% sure which group influenced their behavior.   
This prevents bias, even if you lose some of the sample.   

__Flexible rule__ (keep data according to condition):

Keep the first group the entry was in (based on the earliest date).   
Delete all subsequent records that fall into the other group.   
This keeps more data __but can leave noise__ if the user actually experienced both groups.

In [21]:
display(HTML(f"> Unique Values for 'visitorid' column: {df_orders['visitorid'].nunique()}"))
display(HTML(f"> Total Values for 'visitorid' column: {df_orders['visitorid'].shape[0]}"))

In [22]:
# Get the duplicated values within 'visitorid'
df_orders_group_dup = df_orders.loc[(df_orders['visitorid'].duplicated(keep=False)), :]
df_orders_group_dup

,transactionid,visitorid,date,revenue,group
2,2961555356,4069496402,2019-08-15,10.2,A
15,2550768810,2742574263,2019-08-15,235.0,B
20,2904772834,963407295,2019-08-15,5.6,B
23,4252514150,351125977,2019-08-15,235.6,B
24,1162046357,3234906277,2019-08-15,120.2,B
...,...,...,...,...,...
1182,2682156369,1668030113,2019-08-14,20.8,B
1183,1458356232,963407295,2019-08-14,80.0,A
1184,1329499668,2949041841,2019-08-14,160.7,B
1189,4082620617,393266494,2019-08-14,490.4,B


In [23]:
# Filter those duplicated 'visitorid' that exist in only one group
df_visitors_groups = df_orders_group_dup.groupby(['visitorid', 'group'], observed=False)[['transactionid', 'revenue']].agg({'transactionid': 'count', 'revenue': 'sum'}).reset_index()
df_visitors_groups = df_visitors_groups.rename(columns={'visitorid': 'visitorid', 'group': 'group', 'transactionid': 'transactions', 'revenue': 'revenue'})
df_visitors_groups

,visitorid,group,transactions,revenue
0,8300375,A,1,30.5
1,8300375,B,1,165.7
2,199603092,A,4,311.9
3,199603092,B,1,55.7
4,232979603,A,1,5.9
...,...,...,...,...
183,4186807279,B,2,505.9
184,4256040402,A,7,773.7
185,4256040402,B,4,1977.5
186,4266935830,A,1,1220.2


In [24]:
df_visitors_groups = df_visitors_groups.loc[(df_visitors_groups['transactions'] == 0) & (df_visitors_groups['revenue'] == 0.0), :]
df_visitors_groups

,visitorid,group,transactions,revenue
8,249864742,A,0,0.0
15,366673373,B,0,0.0
18,406208401,A,0,0.0
26,522292794,A,0,0.0
29,611059232,B,0,0.0
...,...,...,...,...
156,3935952912,A,0,0.0
158,3941503500,A,0,0.0
169,3967698036,B,0,0.0
177,4109358064,B,0,0.0


In [25]:
df_orders_group_dup = df_orders_group_dup.loc[~(df_orders_group_dup['visitorid'].isin(df_visitors_groups['visitorid'])), :]
df_orders_group_dup

,transactionid,visitorid,date,revenue,group
2,2961555356,4069496402,2019-08-15,10.2,A
20,2904772834,963407295,2019-08-15,5.6,B
23,4252514150,351125977,2019-08-15,235.6,B
24,1162046357,3234906277,2019-08-15,120.2,B
26,2223239646,199603092,2019-08-15,55.7,A
...,...,...,...,...,...
1182,2682156369,1668030113,2019-08-14,20.8,B
1183,1458356232,963407295,2019-08-14,80.0,A
1184,1329499668,2949041841,2019-08-14,160.7,B
1189,4082620617,393266494,2019-08-14,490.4,B


In [26]:
# Get visitorid belonging to just one group
df_orders = df_orders.loc[~(df_orders['visitorid'].isin(df_orders_group_dup['visitorid'].unique())), :]
df_orders

,transactionid,visitorid,date,revenue,group
0,3667963787,3312258926,2019-08-15,30.4,B
1,2804400009,3642806036,2019-08-15,15.2,B
3,3797467345,1196621759,2019-08-15,155.1,B
4,2282983706,2322279887,2019-08-15,40.5,B
5,182168103,935554773,2019-08-15,35.0,B
...,...,...,...,...,...
1191,3592955527,608641596,2019-08-14,255.7,B
1192,2662137336,3733762160,2019-08-14,100.8,B
1193,2203539145,370388673,2019-08-14,50.1,A
1194,1807773912,573423106,2019-08-14,165.3,A


### 💡 __5. Hypotheses__

##### __5.1__ Prioritize Hypotheses

The hypotheses_us.csv file contains nine hypotheses for increasing online store revenue, with Reach, Impact, Confidence, and Effort specified for each.

In [27]:
df_hypotheses

,hypothesis,reach,impact,confidence,effort
0,Add two new channels for attracting traffic. T...,3,10,8,6
1,Launch your own delivery service. This will sh...,2,5,4,10
2,Add product recommendation blocks to the store...,8,3,7,3
3,Change the category structure. This will incre...,8,3,3,8
4,Change the background color on the main page. ...,3,1,1,1
5,Add a customer review page. This will increase...,3,2,2,3
6,Show banners with current offers and sales on ...,5,3,8,3
7,Add a subscription form to all the main pages....,10,7,8,5
8,Launch a promotion that gives users discounts ...,1,9,9,5


##### __5.1.1__ Apply the ICE framework to prioritize hypotheses. Order them in descending order of priority.

In [28]:
df_hypotheses = ice_prioritization(df_hypotheses, method='effort').sort_values(by='ice_score', ascending=False)
df_hypotheses

,hypothesis,reach,impact,confidence,effort,ice_score
8,Launch a promotion that gives users discounts ...,1,9,9,5,16.200000
0,Add two new channels for attracting traffic. T...,3,10,8,6,13.333333
7,Add a subscription form to all the main pages....,10,7,8,5,11.200000
6,Show banners with current offers and sales on ...,5,3,8,3,8.000000
2,Add product recommendation blocks to the store...,8,3,7,3,7.000000
1,Launch your own delivery service. This will sh...,2,5,4,10,2.000000
5,Add a customer review page. This will increase...,3,2,2,3,1.333333
3,Change the category structure. This will incre...,8,3,3,8,1.125000
4,Change the background color on the main page. ...,3,1,1,1,1.000000


##### __5.1.2__ Apply the RICE framework to prioritize hypotheses. Order them in descending order of priority.

In [29]:
df_hypotheses = ice_prioritization(df_hypotheses, method='rice').sort_values(by='rice_score', ascending=False)
df_hypotheses

,hypothesis,reach,impact,confidence,effort,ice_score,rice_score
7,Add a subscription form to all the main pages....,10,7,8,5,11.200000,112.0
2,Add product recommendation blocks to the store...,8,3,7,3,7.000000,56.0
0,Add two new channels for attracting traffic. T...,3,10,8,6,13.333333,40.0
6,Show banners with current offers and sales on ...,5,3,8,3,8.000000,40.0
8,Launch a promotion that gives users discounts ...,1,9,9,5,16.200000,16.2
3,Change the category structure. This will incre...,8,3,3,8,1.125000,9.0
1,Launch your own delivery service. This will sh...,2,5,4,10,2.000000,4.0
5,Add a customer review page. This will increase...,3,2,2,3,1.333333,4.0
4,Change the background color on the main page. ...,3,1,1,1,1.000000,3.0


##### __5.1.3__ Show how hypothesis prioritization changes when you use RICE instead of ICE. Provide an explanation of the changes.

`LSPL` - The difference is the extra 'reach' factor

1. ICE only balances quality vs cost

    - It looks at how strong an idea is (Impact × Confidence) relative to how hard it is to implement (Effort).
    - It does not care how many users/customers the idea actually touches.

2. RICE adds user scale into the equation

    - By multiplying by Reach, RICE favors ideas that affect a larger audience.
    - This can drastically reorder priorities, because something with moderate impact but high reach can outrank something with very high impact but low reach.

3. The practical effect

    - ICE tends to favor “big wins” (high-impact ideas). Good for internal processes or niche product features where reach is not relevant.
    - RICE tends to favor “broad wins” (ideas that touch more users, even if individually less impactful). Better when you need to decide where to allocate resources for customer-facing features, since reach matters.

##### __5.2__ A/B Test Analysis

##### __5.2.1__ Graph the cumulative income by group. Draw conclusions and make conjectures.

In [30]:
df_orders = df_orders.copy().sort_values(by=['date', 'group'], ascending=[True, True])
df_orders

,transactionid,visitorid,date,revenue,group
55,1144701458,1143609305,2019-08-01,45.3,A
72,477940151,1422773200,2019-08-01,15.8,A
73,1295850747,2947100995,2019-08-01,455.8,A
96,1170105516,2706377257,2019-08-01,140.8,A
98,1962247337,2706377257,2019-08-01,110.7,A
...,...,...,...,...,...
1023,2886170101,930758570,2019-08-31,105.7,B
1024,3845118557,1964413493,2019-08-31,5.5,B
1026,853409662,1390677121,2019-08-31,290.5,B
1037,4149581711,1555048873,2019-08-31,30.2,B


In [31]:
df_orders['cum_income'] = df_orders.groupby('group', observed=True)['revenue'].transform('cumsum')
df_orders

,transactionid,visitorid,date,revenue,group,cum_income
55,1144701458,1143609305,2019-08-01,45.3,A,45.3
72,477940151,1422773200,2019-08-01,15.8,A,61.1
73,1295850747,2947100995,2019-08-01,455.8,A,516.9
96,1170105516,2706377257,2019-08-01,140.8,A,657.7
98,1962247337,2706377257,2019-08-01,110.7,A,768.4
...,...,...,...,...,...,...
1023,2886170101,930758570,2019-08-31,105.7,B,78889.5
1024,3845118557,1964413493,2019-08-31,5.5,B,78895.0
1026,853409662,1390677121,2019-08-31,290.5,B,79185.5
1037,4149581711,1555048873,2019-08-31,30.2,B,79215.7


In [32]:
fig = px.line(df_orders, x="date", y="cum_income", color="group", title="Cumulative Income by Group")
fig.update_xaxes(dtick="D1")
fig.show()

##### __5.2.2__ Graph the cumulative average order size by group. Draw conclusions and make conjectures.

In [33]:
# Get cumulative orders count
df_orders["cum_orders"] = df_orders.groupby("group", observed=False).cumcount() + 1
df_orders

,transactionid,visitorid,date,revenue,group,cum_income,cum_orders
55,1144701458,1143609305,2019-08-01,45.3,A,45.3,1
72,477940151,1422773200,2019-08-01,15.8,A,61.1,2
73,1295850747,2947100995,2019-08-01,455.8,A,516.9,3
96,1170105516,2706377257,2019-08-01,140.8,A,657.7,4
98,1962247337,2706377257,2019-08-01,110.7,A,768.4,5
...,...,...,...,...,...,...,...
1023,2886170101,930758570,2019-08-31,105.7,B,78889.5,544
1024,3845118557,1964413493,2019-08-31,5.5,B,78895.0,545
1026,853409662,1390677121,2019-08-31,290.5,B,79185.5,546
1037,4149581711,1555048873,2019-08-31,30.2,B,79215.7,547


In [34]:
# Get Cumulative Average Order Size
df_orders["cum_avg_order_size"] = df_orders["cum_income"] / df_orders["cum_orders"]
df_orders

,transactionid,visitorid,date,revenue,group,cum_income,cum_orders,cum_avg_order_size
55,1144701458,1143609305,2019-08-01,45.3,A,45.3,1,45.300000
72,477940151,1422773200,2019-08-01,15.8,A,61.1,2,30.550000
73,1295850747,2947100995,2019-08-01,455.8,A,516.9,3,172.300000
96,1170105516,2706377257,2019-08-01,140.8,A,657.7,4,164.425000
98,1962247337,2706377257,2019-08-01,110.7,A,768.4,5,153.680000
...,...,...,...,...,...,...,...,...
1023,2886170101,930758570,2019-08-31,105.7,B,78889.5,544,145.017463
1024,3845118557,1964413493,2019-08-31,5.5,B,78895.0,545,144.761468
1026,853409662,1390677121,2019-08-31,290.5,B,79185.5,546,145.028388
1037,4149581711,1555048873,2019-08-31,30.2,B,79215.7,547,144.818464


In [35]:
fig = px.line(df_orders, x="date", y="cum_avg_order_size", color="group", markers=False, title="Cumulative Average Order Size by Group")
fig.update_xaxes(dtick="D1")
fig.show()

`LSPL` - Seems that group B spends more per day on average than group A. There is a break point in Aug 19, for group B. This should be investigated in order to discard a possible error or confimr it was a result of a relevant event.

##### __5.2.3__ Graph the relative difference in cumulative average order size for Group B compared to Group A. Make conclusions and conjectures.

In [36]:
pivot_aos = df_orders.pivot_table(index="date", columns="group", values="cum_avg_order_size", observed=False).reset_index()
pivot_aos

group,date,A,B
0,2019-08-01,117.183587,67.019405
1,2019-08-02,90.046603,88.675586
2,2019-08-03,85.295335,84.695981
3,2019-08-04,82.356696,87.673320
4,2019-08-05,77.306543,88.892249
...,...,...,...
26,2019-08-27,111.694132,153.176627
27,2019-08-28,110.772273,149.536664
28,2019-08-29,112.064614,147.147501
29,2019-08-30,114.078416,146.183316


In [37]:
pivot_aos["rel_diff"] = pivot_aos["B"] / pivot_aos["A"] - 1
pivot_aos

group,date,A,B,rel_diff
0,2019-08-01,117.183587,67.019405,-0.428082
1,2019-08-02,90.046603,88.675586,-0.015226
2,2019-08-03,85.295335,84.695981,-0.007027
3,2019-08-04,82.356696,87.673320,0.064556
4,2019-08-05,77.306543,88.892249,0.149867
...,...,...,...,...
26,2019-08-27,111.694132,153.176627,0.371394
27,2019-08-28,110.772273,149.536664,0.349947
28,2019-08-29,112.064614,147.147501,0.313059
29,2019-08-30,114.078416,146.183316,0.281428


In [38]:
fig = px.line(pivot_aos, x="date", y="rel_diff", title="Relative Difference in Cumulative Average Order Size (B vs A)")
fig.update_yaxes(tickformat=".1%")
fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="A = B", annotation_position="bottom left")
fig.update_xaxes(dtick="D1")
fig.show()

##### __5.2.4__ Calculate the conversion rate for each group as the ratio of orders to the number of visits each day. Graph the daily conversion rates for the two groups and describe the difference. Draw conclusions and make assumptions.

In [39]:
# Get orders by day and group
daily_orders = (df_orders.groupby(["date", "group"], observed=False)["transactionid"].nunique().reset_index(name="orders"))
daily_orders

,date,group,orders
0,2019-08-01,A,23
1,2019-08-01,B,17
2,2019-08-02,A,19
3,2019-08-02,B,23
4,2019-08-03,A,24
...,...,...,...
57,2019-08-29,B,15
58,2019-08-30,A,9
59,2019-08-30,B,21
60,2019-08-31,A,8


In [40]:
# Merge visits
df_daily_conversion_rate = pd.merge(daily_orders, df_visits, on=["date", "group"], how="right")
df_daily_conversion_rate

,date,group,orders,visits
0,2019-08-01,A,23,719
1,2019-08-02,A,19,619
2,2019-08-03,A,24,507
3,2019-08-04,A,11,717
4,2019-08-05,A,22,756
...,...,...,...,...
57,2019-08-27,B,11,720
58,2019-08-28,B,22,654
59,2019-08-29,B,15,531
60,2019-08-30,B,21,490


In [41]:
df_daily_conversion_rate["conversion_rate"] = df_daily_conversion_rate["orders"] / df_daily_conversion_rate["visits"]
df_daily_conversion_rate

,date,group,orders,visits,conversion_rate
0,2019-08-01,A,23,719,0.031989
1,2019-08-02,A,19,619,0.030695
2,2019-08-03,A,24,507,0.047337
3,2019-08-04,A,11,717,0.015342
4,2019-08-05,A,22,756,0.029101
...,...,...,...,...,...
57,2019-08-27,B,11,720,0.015278
58,2019-08-28,B,22,654,0.033639
59,2019-08-29,B,15,531,0.028249
60,2019-08-30,B,21,490,0.042857


In [42]:
fig = px.line(df_daily_conversion_rate, x="date", y="conversion_rate", color="group", title="Daily Conversion Rate by Group")
fig.update_yaxes(tickformat=".2%")
fig.update_xaxes(dtick="D1")
fig.show()

In [43]:
fig = px.line(title="KDE of Conversion Rate by Group")

for g in df_daily_conversion_rate["group"].unique():
    data = df_daily_conversion_rate.loc[df_daily_conversion_rate["group"] == g, "conversion_rate"].dropna()
    
    # Estimar densidad KDE
    kde = gaussian_kde(data)
    x_range = np.linspace(data.min(), data.max(), 200)
    y_kde = kde(x_range)
    
    # Añadir curva al gráfico
    fig.add_scatter(x=x_range, y=y_kde, mode="lines", name=f"Group {g}")

fig.update_layout(xaxis_title="Conversion Rate", yaxis_title="Density")

fig.show()

In [44]:
cumulative_visits = df_visits.groupby(['date', 'group'], observed=False).agg(visits=('visits', 'sum'))
cumulative_visits

visits
date       group        
2019-08-01 A         719
           B         713
2019-08-02 A         619
           B         581
2019-08-03 A         507
...                  ...
2019-08-29 B         531
2019-08-30 A         490
           B         490
2019-08-31 A         699
           B         718

[62 rows x 1 columns]

In [45]:
cumulative_visits = cumulative_visits.groupby('group', observed=False).cumsum().reset_index()
cumulative_visits

,date,group,visits
0,2019-08-01,A,719
1,2019-08-01,B,713
2,2019-08-02,A,1338
3,2019-08-02,B,1294
4,2019-08-03,A,1845
...,...,...,...
57,2019-08-29,B,17708
58,2019-08-30,A,18037
59,2019-08-30,B,18198
60,2019-08-31,A,18736


In [46]:
cumulative_visits

,date,group,visits
0,2019-08-01,A,719
1,2019-08-01,B,713
2,2019-08-02,A,1338
3,2019-08-02,B,1294
4,2019-08-03,A,1845
...,...,...,...
57,2019-08-29,B,17708
58,2019-08-30,A,18037
59,2019-08-30,B,18198
60,2019-08-31,A,18736


In [47]:
df_orders = df_orders.merge(cumulative_visits, on=['date', 'group'])
df_orders = df_orders.rename(columns={'visits': 'cum_visits'})
df_orders

,transactionid,visitorid,date,revenue,group,cum_income,cum_orders,cum_avg_order_size,cum_visits
0,1144701458,1143609305,2019-08-01,45.3,A,45.3,1,45.300000,719
1,477940151,1422773200,2019-08-01,15.8,A,61.1,2,30.550000,719
2,1295850747,2947100995,2019-08-01,455.8,A,516.9,3,172.300000,719
3,1170105516,2706377257,2019-08-01,140.8,A,657.7,4,164.425000,719
4,1962247337,2706377257,2019-08-01,110.7,A,768.4,5,153.680000,719
...,...,...,...,...,...,...,...,...,...
1011,2886170101,930758570,2019-08-31,105.7,B,78889.5,544,145.017463,18916
1012,3845118557,1964413493,2019-08-31,5.5,B,78895.0,545,144.761468,18916
1013,853409662,1390677121,2019-08-31,290.5,B,79185.5,546,145.028388,18916
1014,4149581711,1555048873,2019-08-31,30.2,B,79215.7,547,144.818464,18916


In [48]:
df_orders['cum_conversion'] = df_orders['cum_orders'] / df_orders['cum_visits']
df_orders

,transactionid,visitorid,date,revenue,group,cum_income,cum_orders,cum_avg_order_size,cum_visits,cum_conversion
0,1144701458,1143609305,2019-08-01,45.3,A,45.3,1,45.300000,719,0.001391
1,477940151,1422773200,2019-08-01,15.8,A,61.1,2,30.550000,719,0.002782
2,1295850747,2947100995,2019-08-01,455.8,A,516.9,3,172.300000,719,0.004172
3,1170105516,2706377257,2019-08-01,140.8,A,657.7,4,164.425000,719,0.005563
4,1962247337,2706377257,2019-08-01,110.7,A,768.4,5,153.680000,719,0.006954
...,...,...,...,...,...,...,...,...,...,...
1011,2886170101,930758570,2019-08-31,105.7,B,78889.5,544,145.017463,18916,0.028759
1012,3845118557,1964413493,2019-08-31,5.5,B,78895.0,545,144.761468,18916,0.028812
1013,853409662,1390677121,2019-08-31,290.5,B,79185.5,546,145.028388,18916,0.028864
1014,4149581711,1555048873,2019-08-31,30.2,B,79215.7,547,144.818464,18916,0.028917


In [49]:
fig = px.line(df_orders, x="date", y="cum_conversion", color="group", title="Cumulative Conversion Rate by Group")
fig.update_yaxes(tickformat=".2%")
fig.update_xaxes(dtick="D1")
fig.show()

`LSPL`: 
- Group B performs better in conversion than A, both in the overall distribution (KDE) and in the cumulative rate.
- The daily signal is noisy, so relying solely on the daily conversion graph can be misleading.
- The fact that B maintains an advantage in the cumulative rate suggests that the difference is not due to chance.

Next step: confirm with a statistical significance test to validate whether the difference is statistically reliable.

##### __5.2.5__  Draw a scatter plot of the number of orders per user. Make conclusions and conjectures.

In [50]:
df_orders = df_orders.loc[:, ['transactionid', 'visitorid', 'date', 'revenue', 'group']]
df_orders

,transactionid,visitorid,date,revenue,group
0,1144701458,1143609305,2019-08-01,45.3,A
1,477940151,1422773200,2019-08-01,15.8,A
2,1295850747,2947100995,2019-08-01,455.8,A
3,1170105516,2706377257,2019-08-01,140.8,A
4,1962247337,2706377257,2019-08-01,110.7,A
...,...,...,...,...,...
1011,2886170101,930758570,2019-08-31,105.7,B
1012,3845118557,1964413493,2019-08-31,5.5,B
1013,853409662,1390677121,2019-08-31,290.5,B
1014,4149581711,1555048873,2019-08-31,30.2,B


In [51]:
df_orders_per_user_group = (df_orders.groupby(["visitorid", 'group'], observed=False)["transactionid"].nunique().reset_index(name="orders"))
df_orders_per_user_group = df_orders_per_user_group.loc[(df_orders_per_user_group['orders'] != 0), :]
df_orders_per_user_group

,visitorid,group,orders
1,5114589,B,1
3,6958315,B,1
4,11685486,A,1
7,39475350,B,1
9,47206413,B,1
...,...,...,...
1936,4259830713,A,1
1938,4278982564,A,1
1941,4279090005,B,1
1943,4281247801,B,1


In [52]:
fig = px.scatter(df_orders_per_user_group, x="visitorid", y="orders", color="group", opacity=0.33, title="Orders per User by Group",
                 labels={"orders": "Number of Orders", "visitorid": "User ID"})
fig.update_xaxes(dtick=100000000, tickformat="d", tickangle=90, rangemode='tozero')
fig.show()

`LSPL` - Conversion behavior is consistent across groups: most users buy only once, regardless of group assignment.   
Multi-order users are rare: these could be legitimate high-value customers, or in some cases anomalies.

##### __5.2.6__ Calculates the 95th and 99th percentiles of the number of orders per user. Defines the point at which a data point becomes an anomaly.

In [53]:
df_orders_per_user = df_orders.groupby("visitorid")["transactionid"].nunique().reset_index(name="orders")
df_orders_per_user

,visitorid,orders
0,5114589,1
1,6958315,1
2,11685486,1
3,39475350,1
4,47206413,1
...,...,...
968,4259830713,1
969,4278982564,1
970,4279090005,1
971,4281247801,1


In [54]:
p95 = df_orders_per_user["orders"].quantile(0.95)
p99 = df_orders_per_user["orders"].quantile(0.99)

display(HTML(f"> Percentile 95: <b>{p95}</b>"))
display(HTML(f"> Percentile 99: <b>{p99}</b>"))  

In [55]:
# If you use the 95th percentile → anomaly = users with orders > p95.
display(HTML(f"> Anomaly if p95 is used = users with orders > p95"))
df_orders_per_user.loc[(df_orders_per_user['orders'] > 1), :] 

,visitorid,orders
55,249864742,3
82,366673373,2
94,406208401,2
118,522292794,2
138,611059232,3
...,...,...
897,3935952912,2
901,3941503500,2
908,3967698036,3
937,4109358064,2


In [56]:
# If you use the 99th percentile → anomaly = users with orders > p99.
display(HTML(f"> Anomaly if p99 is used = users with orders > p99"))
df_orders_per_user.loc[(df_orders_per_user['orders'] > 2), :] 

,visitorid,orders
55,249864742,3
138,611059232,3
478,2108163459,3
632,2742574263,3
687,2988190573,3
890,3908431265,3
908,3967698036,3


##### __5.2.7__ Draw a scatter plot of the order prices. Make conclusions and conjectures.

In [57]:
fig = px.scatter(df_orders, x="transactionid", y="revenue", color="group", title="Order Prices by Group",
                 labels={"transactionid": "Order ID", "revenue": "Order Price"})
fig.update_yaxes(dtick=1000, title="Revenue", rangemode="tozero")
fig.update_xaxes(dtick=100000000, tickformat="d", tickangle=90, rangemode="tozero")
fig.show()

`LSPL` - Most orders are clustered at lower revenue (less than 1K), higher valued orders are outliers. Outliers artificially inflate the average order size, that's why cumulative averages and statistical tests are run with and without filtering anomalies.

##### __5.2.8__ Calculate the 95th and 99th percentiles of order prices. Defines the point at which a data point becomes an anomaly.

In [58]:
p95 = df_orders["revenue"].quantile(0.95)
p99 = df_orders["revenue"].quantile(0.99)

display(HTML(f"> Percentile 95: <b>{p95}</b>"))
display(HTML(f"> Percentile 99: <b>{p99}</b>"))  

In [59]:
# If you use the 95th percentile → anomaly = users with prices > p95.
display(HTML(f"> Anomaly if p95 is used = transactions with prices > p95"))
df_orders.loc[(df_orders['revenue'] > 414.275), :] 

,transactionid,visitorid,date,revenue,group
2,1295850747,2947100995,2019-08-01,455.8,A
10,1798931686,1211585711,2019-08-01,440.3,A
59,1192036073,2953427180,2019-08-02,590.1,B
66,573486797,3118156181,2019-08-02,475.5,B
134,1867784682,2247666522,2019-08-04,485.3,B
...,...,...,...,...,...
945,3964942202,358944393,2019-08-29,660.5,A
975,1234615529,2279926127,2019-08-30,705.3,B
986,1538887006,3372131337,2019-08-30,555.2,B
989,2894657060,204675465,2019-08-30,510.5,B


In [60]:
# If you use the 99th percentile → anomaly = transactions with prices > p99.
display(HTML(f"> Anomaly if p99 is used = transactions with prices > p99"))
df_orders.loc[(df_orders['revenue'] > 830.3), :] 

,transactionid,visitorid,date,revenue,group
294,2420050534,4003628586,2019-08-08,905.8,B
310,3603576309,4133034833,2019-08-09,1050.0,A
404,316924019,148427295,2019-08-12,1015.9,A
405,1348774318,1164614297,2019-08-12,1025.8,A
438,666610489,1307669133,2019-08-13,1425.8,A
529,3936777065,2108080724,2019-08-15,3120.1,B
635,590470918,1920142716,2019-08-19,19920.4,B
696,1347999392,887908475,2019-08-21,930.0,A
882,3668308183,888512513,2019-08-27,1335.6,B


##### __5.2.9__ Find the statistical significance of the difference in conversion between groups using the raw data. Make conclusions and conjectures.

In [86]:
orders_per_group = df_orders.groupby("group", observed=False)["visitorid"].nunique()
orders_per_group.name = 'orders'
orders_per_group

group
A    445
B    528
Name: orders, dtype: int64

In [78]:
visits_per_group = df_visits.groupby("group", observed=False)["visits"].sum()
visits_per_group

group
A    18736
B    18916
Name: visits, dtype: int64

In [79]:
successes = np.array([orders_per_group["A"], orders_per_group["B"]])
successes

array([445, 528])

In [80]:
trials = np.array([visits_per_group["A"], visits_per_group["B"]])
trials

array([18736, 18916])

In [89]:
stats, p_value = proportions_ztest(successes, trials, alternative="two-sided")
display(HTML(f"Z-statistic: {stats}"))
display(HTML(f"p-value: {p_value}"))

In [90]:
cr_per_group = pd.concat([orders_per_group, visits_per_group], axis=1)
cr_per_group.columns = ['orders', 'visits']
cr_per_group['conversion_rate'] = cr_per_group['orders'] / cr_per_group['visits']
cr_per_group

,orders,visits,conversion_rate
group,,,
A,445,18736,0.023751
B,528,18916,0.027913


In [ ]:
# Null hypothesis (H₀): Conversion rates of group A and B are equal.
# Alternative hypothesis (H₁): Conversion rates of group A and B are different.
if p_value <= 0.05:
    display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>rejected</b>, meaning there is enough statistical evidence that conversion rate between Group A and Group B are <b>different</b>."))
    if cr_per_group.loc['A', 'conversion_rate'] > cr_per_group.loc['B', 'conversion_rate']:
        display(HTML(f"> Group <b>A</b> has a higher conversion rate ({cr_group_A:.4%}) than Group B ({cr_group_B:.4%})."))
    else:
        display(HTML(f"> Group <b>B</b> has a higher conversion rate ({cr_group_B:.4%}) than Group A ({cr_group_A:.4%})."))
else:
    display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>not rejected</b>, meaning there is not enough statistical evidence that conversion rate between Group A and Group B are different."))

##### __5.2.10__ Find the statistical significance of the difference in average order size between groups using the raw data. Make conclusions and conjectures.

In [92]:
aos_group_A = df_orders.loc[(df_orders["group"] == "A"), "revenue"]
aos_group_A

0       45.3
1       15.8
2      455.8
3      140.8
4      110.7
       ...  
994    270.4
995    155.8
996    205.1
997     45.2
998     55.4
Name: revenue, Length: 468, dtype: float64

In [93]:
aos_group_B = df_orders.loc[(df_orders["group"] == "B"), "revenue"]
aos_group_B

23       40.2
24      140.4
25       90.5
26       50.8
27       40.2
        ...  
1011    105.7
1012      5.5
1013    290.5
1014     30.2
1015    435.5
Name: revenue, Length: 548, dtype: float64

In [95]:
# Mann-whitney U test
u_stats, u_p_value = st.mannwhitneyu(aos_group_A, aos_group_B, alternative="two-sided")
display(HTML(f"U-statistic: {u_stats}"))
display(HTML(f"p-value: {u_p_value}"))

In [96]:
# Null hypothesis (H₀): Average Order Size of group A and B are equal.
# Alternative hypothesis (H₁): Average Order Size of group A and B are different.
if u_p_value <= 0.05:
    display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>rejected</b>, meaning there is enough statistical evidence that AOS between Group A and Group B are <b>different</b>."))
else:
    display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>not rejected</b>, meaning there is not enough statistical evidence that AOS between Group A and Group B are different."))

`LSPL`
- The cumulative AOS plot shows Group B consistently above Group A → looks like B has bigger orders.
- But the Mann–Whitney U test gave p = 0.86223 → very high p-value → no evidence of a real difference between groups.

That means:

- Even though Group B’s line is higher in the cumulative plot, we can’t rule out that the difference is just due to random variation.
- There’s no strong evidence that Group B genuinely generates higher order sizes.

Even though Group B’s average order size looks higher in the plot, the statistical test shows no significant difference (p=0.86). The apparent gap is likely due to randomness or noise in the data, not a real effect.

##### __5.2.11__ Find the statistical significance of the difference in conversion between the groups using the filtered data. Make conclusions and conjectures.